<a href="https://colab.research.google.com/github/cosmina98/PhD/blob/main/Work_in_progress/MessPass_GAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [2]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
!pip install plotnine --quiet
!pip install torch_geometric
! install torch_geometric
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.12.1+cu113
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 467 kB 4.2 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.1.0.post1-py3-none-any.whl size=689859 sha256=07fe4fa2f2fca4a27dadb19172dacb299d09adb07533b6756e1455c4b086ff66
  Stored in directory: /root/.cache/pip/wheels/d1/cb/43/f7f2e472de4d7cff31bceddadc36d634e1e545fbc17961c282
Successfully built torch-geometric
install: missing destination file operand after 'torch_geometric'
Try 'install --help' for more information.
     |████████████████████████████████| 7.9 MB 2.7 MB/s 
     |████████████████████████████████| 3.5 MB 2.5 MB/s 


In [3]:
import torch_geometric
from torch_geometric.datasets import Planetoid
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

In [56]:
import torch_geometric 
from typing import Callable, List, Optional, Tuple
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch import Tensor
from torch.optim import Optimizer
from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv,ChebConv,GATConv
from torch_geometric.utils import softmax, add_remaining_self_loops
#from torch_geometric.utils import accuracy
from typing_extensions import Literal, TypedDict
from torch_geometric.utils import get_laplacian
import numpy as np
import pandas as pd
from plotnine import ggplot, geom_line, aes, xlab, theme, element_blank, ggtitle
import scipy.sparse as sparse
from sklearn.model_selection import KFold
from torch_geometric.utils import to_scipy_sparse_matrix
from dataclasses import dataclass

In [55]:
# libary imports 
import os
import torch
from torch import nn
import torchvision
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision.datasets as datasets

from sklearn import metrics
from sklearn.metrics import accuracy_score

from sklearn import decomposition
from sklearn import manifold
from tqdm.notebook import trange, tqdm

import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import matplotlib.pyplot as plt
import numpy as np
import copy
import random
import time
import pandas as pd

import itertools
from functools import reduce
import operator
import torch.utils.data as data_utils
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import multilabel_confusion_matrix
from torch.utils.data import Subset

from torch.optim.adadelta import Adadelta
from torch.optim.adagrad import Adagrad
from torch.optim.adam import Adam
from torch.optim.adamax import Adamax
from torch.optim.adamw import AdamW
from torch.optim.optimizer import Optimizer
from torch.optim.sgd import SGD

import torch_geometric 
from typing import Callable, List, Optional, Tuple
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch import Tensor
from torch.optim import Optimizer
from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv,ChebConv
from typing_extensions import Literal, TypedDict
from torch_geometric.utils import get_laplacian
from torch_geometric.nn import MessagePassing

import numpy as np
import pandas as pd
from plotnine import ggplot, geom_line, aes, xlab, theme, element_blank, ggtitle
import scipy.sparse as sparse
from sklearn.model_selection import KFold
from torch_geometric.utils import to_scipy_sparse_matrix
from dataclasses import dataclass
import torch_geometric.utils
#from torch_geometric.utils import accuracy


In [6]:
device = torch.device('cuda' if torch.cuda.is_available()  else 'cpu')


#Dataset imports

In [7]:
dataset=list(range(3))
i=0
for dataset_name in ['Cora','CiteSeer','PubMed']:
    dataset[i] = Planetoid('/tmp/dataset_name', name=dataset_name)
    num_nodes = dataset[i].data.num_nodes
    num_edges = dataset[i].data.num_edges // 2
    train_len = dataset[i][0].train_mask.sum()
    val_len = dataset[i][0].val_mask.sum()
    test_len = dataset[i][0].test_mask.sum()
    other_len = num_nodes - train_len - val_len - test_len
    print(f"Dataset: {dataset[i].name}")
    print(f"Num. nodes: {num_nodes} (train={train_len}, val={val_len}, test={test_len}, other={other_len})")
    print(f"Num. edges: {num_edges}")
    print(f"Num. node features: {dataset[i].num_node_features}")
    print(f"Num. classes: {dataset[i].num_classes}")
    print(f"Dataset len.: {dataset[i].len()}")
    i=i+1
    print(" ")


Processing...
Done!


Dataset: Cora
Num. nodes: 2708 (train=140, val=500, test=1000, other=1068)
Num. edges: 5278
Num. node features: 1433
Num. classes: 7
Dataset len.: 1
 


Processing...
Done!


Dataset: CiteSeer
Num. nodes: 3327 (train=120, val=500, test=1000, other=1707)
Num. edges: 4552
Num. node features: 3703
Num. classes: 6
Dataset len.: 1
 


Processing...


Dataset: PubMed
Num. nodes: 19717 (train=60, val=500, test=1000, other=18157)
Num. edges: 44324
Num. node features: 500
Num. classes: 3
Dataset len.: 1
 


Done!


# Training and testing targets for each dataset

In [7]:
d={0:{}, 1:{}, 2:{}}
for i in range(3):
    data=dataset[i]
    X_train=data[0].x[data[0].train_mask]
    d[i]['X_train']=X_train
    y_train=data[0].y[data[0].train_mask]
    d[i]['y_train']=y_train
    X_test=data[0].x[data[0].test_mask]
    d[i]['X_test']=X_test
    y_test=data[0].y[data[0].test_mask]
    d[i]['y_test']=y_test
    X_val=data[0].x[data[0].val_mask]
    d[i]['X_val']=X_val
    y_val=data[0].y[data[0].val_mask]
    d[i]['y_val']=X_val


#Message Passing by inheriting MessagePassing class

In [8]:
from torch_scatter import scatter_add

dataset[0].data
x=dataset[0].data.x
edge_index=dataset[0].data.edge_index
x_j = x[edge_index[0]]  # Source node features [num_edges, num_features]
x_i = x[edge_index[1]]  # Target node features [num_edges, num_features]

In [9]:

class GATConv(MessagePassing):
    def __init__(self, in_channels, out_channels, alpha, drop_prob=0.0):
        super().__init__(aggr="add")
        self.drop_prob = drop_prob
        self.lin = nn.Linear(in_channels, out_channels, bias=False)
        self.a = nn.Parameter(torch.zeros(size=(2*out_channels, 1)))
        self.leakrelu = nn.LeakyReLU(alpha)
        nn.init.xavier_uniform_(self.a)

    def forward(self, x, edge_index):
        edge_index, _ = add_remaining_self_loops(edge_index)
        #  Calculation  Wh
        h = self.lin(x)
        #  Start message propagation 
        h_prime = self.propagate(edge_index, x=h)
        return h_prime

    def message(self, x_i, x_j, edge_index_i):
        #  Calculation a(Wh_i || wh_j)
        e = torch.matmul((torch.cat([x_i, x_j], dim=-1)), self.a)
        e = self.leakrelu(e)
        alpha = softmax(e, edge_index_i)
        alpha = F.dropout(alpha, self.drop_prob, self.training)
        return x_j * alpha


if __name__ == "__main__":
    conv = GATConv(in_channels=3, out_channels=3, alpha=0.2)
    x = torch.rand(4, 3)
    edge_index = torch.tensor(
        [[0, 1, 1, 2, 0, 2, 0, 3], [1, 0, 2, 1, 2, 0, 3, 0]], dtype=torch.long)
    x = conv(x, edge_index)
    print(x.shape)

torch.Size([4, 3])


In [37]:
conv = GATConv(x.shape[1], 256,alpha=0.2)
m=conv(x, edge_index)
m

tensor([[-0.0955, -0.0324,  0.1994,  ...,  0.1117,  0.0324,  0.0495],
        [-0.1058, -0.0364,  0.2034,  ...,  0.1157,  0.0358,  0.0597],
        [-0.1058, -0.0364,  0.2034,  ...,  0.1157,  0.0358,  0.0597],
        [-0.0831, -0.0275,  0.1979,  ...,  0.1078,  0.0291,  0.0370]],
       grad_fn=<ScatterAddBackward0>)

#Message passing as torch.nn.moduke

In [39]:
torch.manual_seed(0)
from torch_geometric.utils import softmax, add_remaining_self_loops

class GATLayer(nn.Module):
    
    def __init__(self, in_channels, out_channels, alpha, dropout=0.0):
        super().__init__()  
        #initialisation
        self.lin = nn.Linear(in_channels, out_channels, bias=False)
        self.flow = 'source_to_target'
        self.out_channels=out_channels
        self.drop_prob = dropout
        self.leakrelu = nn.LeakyReLU(alpha)
        self.a = nn.Parameter(torch.zeros(size=(2*out_channels, 1)))
        nn.init.xavier_uniform_(self.a)
        self.reset_parameters()

    def sum_aggregate(self,x,edge_index,num_nodes):
        row,col=edge_index
        tmp = torch.index_select(x, 0, row) # shape [num_edges, embed_size ]
        index2 = col.expand(( self.out_channels, col.size(0))).T
        # same result by using torch.scatter_add
        aggr2 = torch.zeros(num_nodes, self.out_channels, dtype=tmp.dtype).scatter_add(0, index2, tmp)
        return aggr2

    def reset_parameters(self):
        self.lin.reset_parameters()
        self.bias.data.zero_()

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        num_nodes=len(x)
        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_remaining_self_loops(edge_index)

        # Step 2: Linearly transform node feature matrix.
        z = self.lin(x)

        # Step 4: Start propagating messages.
        out=self.collect(z,edge_index)
        x_j=out['x_j']
        x_i= out['x_i']
        edge_index_i= out['edge_index_i']
        
        #Step 5 Compute the attention score 
        e = self.leakrelu(torch.matmul((torch.cat([x_i, x_j], dim=-1)), self.a))
        
        #Step 6 Normalise the attention score
        alpha = softmax(e, edge_index_i)
        alpha = F.dropout(alpha, self.drop_prob, self.training)
        
        #Step 7 Compute the final message
        out = self.message(x_j,alpha)

        #Step 8 Aggregate messages 
        out= self.sum_aggregate(out,edge_index, num_nodes)


        return out

    def collect(self ,x,edge_index):
        i, j = (1, 0) if self.flow == 'source_to_target' else (0, 1)
        out={}
        # 2. construct message x_j, x_i. Both with shape [num_edge, embed_size]
        out['x_j'] = x.index_select(0, edge_index[i]) 
        out['x_i'] = x.index_select(0, edge_index[j])
        out['edge_index_i'] = edge_index[i] # Source node edges 
        out['edge_index_j'] = edge_index[j]   # Target node edges 
        return out  

   
    def message(self, x_j,alpha):
        # x_j has shape [E, out_channels]

        return x_j * alpha

    #'not needed'
    def update(self, aggr_out):
    # aggr_out has shape [num_nodes, out_channels]

    # Step 5: Return new node embeddings.
        return aggr_out

In [40]:
conv = GATLayer(x.shape[1], 256,alpha=0.2)
m=conv(x, edge_index)
m

tensor([[ 0.0545, -0.1748, -0.2349,  ...,  0.0624, -0.2049, -0.2106],
        [ 0.0403, -0.1284, -0.1723,  ...,  0.0458, -0.1508, -0.1547],
        [ 0.0403, -0.1284, -0.1723,  ...,  0.0458, -0.1508, -0.1547],
        [ 0.0284, -0.0927, -0.1250,  ...,  0.0330, -0.1081, -0.1118]],
       grad_fn=<AddBackward0>)

In [ ]:
class GAT(torch.nn.Module):
    def __init__(self,num_features,num_classes):
        super(GAT, self).__init__()
        self.hid = 8
        self.in_head = 8
        self.out_head = 1

        self.conv1 = GATLayer(num_features, self.hid, dropout=0.6,alpha=0.2)
        self.conv2 = GATLayer(self.hid*self.in_head, num_classes,
                             dropout=0.6,alpha=0.2)

    def forward(self, x, edge_index):
                
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        
        return F.log_softmax(x, dim=1)

In [52]:
print(GAT(dataset[0].num_node_features, dataset[0].num_classes))
print(GAT(dataset[1].num_node_features, dataset[1].num_classes))
print(GAT(dataset[2].num_node_features, dataset[2].num_classes))

GAT(
  (conv1): GATLayer(
    (lin): Linear(in_features=1433, out_features=8, bias=False)
    (leakrelu): LeakyReLU(negative_slope=0.2)
  )
  (conv2): GATLayer(
    (lin): Linear(in_features=64, out_features=7, bias=False)
    (leakrelu): LeakyReLU(negative_slope=0.2)
  )
)
GAT(
  (conv1): GATLayer(
    (lin): Linear(in_features=3703, out_features=8, bias=False)
    (leakrelu): LeakyReLU(negative_slope=0.2)
  )
  (conv2): GATLayer(
    (lin): Linear(in_features=64, out_features=6, bias=False)
    (leakrelu): LeakyReLU(negative_slope=0.2)
  )
)
GAT(
  (conv1): GATLayer(
    (lin): Linear(in_features=500, out_features=8, bias=False)
    (leakrelu): LeakyReLU(negative_slope=0.2)
  )
  (conv2): GATLayer(
    (lin): Linear(in_features=64, out_features=3, bias=False)
    (leakrelu): LeakyReLU(negative_slope=0.2)
  )
)
